In [20]:
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer,TrainingArguments,Trainer


model_id = "nadika/grievance_classification"  
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)


In [21]:
import pandas as pd
df=pd.read_csv('../data/incremental.csv')
df.head()

,label,text
0,अर्थ सबन्धी,मैले मेरो बुवाको २४ मार्च २०२१ मा मेरो डीईसको ...
1,अर्थ सबन्धी,"suru maa 11.5 theyo interest , ra aaile 14.35 ..."
2,अर्थ सबन्धी,खोइ त हेलो सरकार !!!!! मेरो कोरोना बिमा को भुक...
3,अर्थ सबन्धी,मेरो किस्पाङ पशु तथा कृषि फार्म प्रा.लि. को २०...
4,अर्थ सबन्धी,इन्सुरंस कम्पनि हरुले कोरोना बिमाको भुक्तानी ह...


In [22]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])
labels = df['label'].tolist()
df['label']


0       0
1       0
2       0
3       0
4       0
       ..
295     9
296     1
297    12
298     7
299     9
Name: label, Length: 300, dtype: int64

In [23]:
df

,label,text
0,0,मैले मेरो बुवाको २४ मार्च २०२१ मा मेरो डीईसको ...
1,0,"suru maa 11.5 theyo interest , ra aaile 14.35 ..."
2,0,खोइ त हेलो सरकार !!!!! मेरो कोरोना बिमा को भुक...
3,0,मेरो किस्पाङ पशु तथा कृषि फार्म प्रा.लि. को २०...
4,0,इन्सुरंस कम्पनि हरुले कोरोना बिमाको भुक्तानी ह...
...,...,...
295,9,नागरिक aap मा भएका document offline Pani देखिन...
296,1,नेपाल सरकारले covid 19 को 2 वटा खोप लगायको सर्...
297,12,स्वास्थ्य सेवा दिनु भनेको एउटा विशुद्ध सेवा मु...
298,7,वानेश्वर तिनकुने गौरी मार्गमा एक हप्ता देखी फो...


In [24]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import torch

# Load the dataset
# df = pd.read_csv('../data/incremental.csv')

# Assuming the CSV has two columns: 'text' for the input text and 'label' for the target labels
texts = df['text'].tolist()
labels = df['label'].tolist()

# Tokenize the text
tokenizer = AutoTokenizer.from_pretrained("nadika/grievance_classification")

class GrievanceDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Prepare the dataset
encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
dataset = GrievanceDataset(encodings, labels)

# Split the dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)


In [25]:
from transformers import AutoModelForSequenceClassification

model_id = "nadika/grievance_classification"
model = AutoModelForSequenceClassification.from_pretrained(model_id)


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)


/home/mahima/Documents/GrievanceRecognition/venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
df.head()

,label,text
0,0,मैले मेरो बुवाको २४ मार्च २०२१ मा मेरो डीईसको ...
1,0,"suru maa 11.5 theyo interest , ra aaile 14.35 ..."
2,0,खोइ त हेलो सरकार !!!!! मेरो कोरोना बिमा को भुक...
3,0,मेरो किस्पाङ पशु तथा कृषि फार्म प्रा.लि. को २०...
4,0,इन्सुरंस कम्पनि हरुले कोरोना बिमाको भुक्तानी ह...


In [28]:
from sklearn.metrics import accuracy_score
import numpy as np

epochs = 3

for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    model.eval()
    predictions, true_vals = [], []
    for batch in val_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        
        logits = outputs.logits
        predictions.append(logits.cpu().numpy())
        true_vals.append(batch['category'].cpu().numpy())
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    preds_flat = np.argmax(predictions, axis=1).flatten()
    true_vals_flat = true_vals.flatten()
    val_accuracy = accuracy_score(true_vals_flat, preds_flat)
    print(f'Validation Accuracy Epoch {epoch + 1}: {val_accuracy}')

# Save the model
model_path = "/mnt/data/grievance_classification_finetuned"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacty of 3.81 GiB of which 93.06 MiB is free. Including non-PyTorch memory, this process has 3.71 GiB memory in use. Of the allocated memory 3.35 GiB is allocated by PyTorch, and 289.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF